In [1]:
import sys
sys.path.append('../../')
import time
import argparse
import os
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from utils import find_image_file
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import torch.utils.data as Data
from importlib import import_module
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics

In [2]:
train_node = pd.read_csv("../../data/Process/train_node.csv")
test_node = pd.read_csv("../../data/Process/test_node.csv")
test_node

,id,amino_acid_seq:String,~label,label,nid
0,b3064,MRVLGIETSCDETGIAIYDDEKGLLANQLYSQVKLHADYGGVVPEL...,0,Gene,6
1,b3594,MSNTYQKRKASKEYGLYNQCKKLNDDELFRLLDDHNSLKRISSARV...,0,Gene,1171
2,b2918,MINEATLAESIRRLRQGERATLAQAMTLVESRHPRHQALSTQLLDA...,0,Gene,99
3,b4015,MKTRTQQIEELQKEWTQPRWEGITRPYSAEDVVKLRGSVNPECTLA...,0,Gene,927
4,b3176,MSNRKYFGTDGIRGRVGDAPITPDFVLKLGWAAGKVLARHGSRKII...,0,Gene,314
...,...,...,...,...,...
905,b4396,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,1,Gene,968
906,b3943,MSAAGKSNPLAISGLVVLTLIWSYSWIFMKQVTSYIGAFDFTALRC...,0,Gene,877
907,b1204,MTIKLIVGLANPGAEYAATRHNAGAWFVDLLAERLRAPLREEAKFF...,0,Gene,2834
908,b2224,MKNCVIVSAVRTAIGSFNGSLASTSAIDLGATVIKAAIERAKIDSQ...,0,Gene,2545


In [3]:
env_path_list = []
find_image_file('../../../../data/ESM_Protein_Embedding',env_path_list)
label = [x.split('/')[-1].replace('.pt','') for x in env_path_list]
rawdata = pd.DataFrame([env_path_list,label],index=['path','label']).T
rawdata

,path,label
0,../../../../data/ESM_Protein_Embedding/b2162.pt,b2162
1,../../../../data/ESM_Protein_Embedding/b1777.pt,b1777
2,../../../../data/ESM_Protein_Embedding/b2709.pt,b2709
3,../../../../data/ESM_Protein_Embedding/b1201.pt,b1201
4,../../../../data/ESM_Protein_Embedding/b1565.pt,b1565
...,...,...
4317,../../../../data/ESM_Protein_Embedding/b2835.pt,b2835
4318,../../../../data/ESM_Protein_Embedding/b3133.pt,b3133
4319,../../../../data/ESM_Protein_Embedding/b0480.pt,b0480
4320,../../../../data/ESM_Protein_Embedding/b3774.pt,b3774


In [4]:
train_data = pd.merge(train_node,rawdata,left_on="id",right_on="label")
train_data['embedding'] = train_data.path.apply(lambda x :torch.load(x)['mean_representations'][0].tolist())
test_data = pd.merge(test_node,rawdata,left_on="id",right_on="label")
test_data['embedding'] = test_data.path.apply(lambda x :torch.load(x)['mean_representations'][0].tolist())
train_data = train_data[["id","embedding","~label"]]
test_data = test_data[["id","embedding","~label"]]
test_data

,id,embedding,~label
0,b3064,"[-0.01100315060466528, -0.005879893433302641, ...",0
1,b3594,"[-0.013980396091938019, 0.00847859587520361, -...",0
2,b2918,"[-0.008102411404252052, -0.011242958717048168,...",0
3,b4015,"[-0.019887518137693405, -0.012007234618067741,...",0
4,b3176,"[-0.016953254118561745, -0.001588500221259892,...",0
...,...,...,...
905,b4396,"[-0.011818330734968185, -0.011900028213858604,...",1
906,b3943,"[0.001066086464561522, -0.0026041993405669928,...",0
907,b1204,"[-0.012061668559908867, -0.0053014978766441345...",0
908,b2224,"[-0.013435409404337406, -0.0078052193857729435...",0


In [5]:
class MyDataSet(Data.Dataset):
    def __init__(self, data):
        self.x = data['embedding'].values
        self.y = data['~label'].values

    def __getitem__(self, index):
        return np.array(self.x[index],dtype=np.float32).reshape(1,1280),self.y[index]

    def __len__(self):
        return len(self.x)
train_dataset = MyDataSet(train_data)
test_dataset = MyDataSet(test_data)

batch_size=100
train_iter = Data.DataLoader(train_dataset, batch_size=batch_size)
test_iter = Data.DataLoader(test_dataset, batch_size=batch_size)

In [6]:
train_dataset[0][0].shape

(1, 1280)

In [8]:
model_name ='TextRNN_Att'  

x = import_module(f'models.{model_name}')
config = x.Config()
model = x.Model(config).to(config.device)
print(model.parameters)

<bound method Module.parameters of Model(
  (lstm): LSTM(1280, 512, num_layers=3, batch_first=True, dropout=0.5, bidirectional=True)
  (tanh1): Tanh()
  (tanh2): Tanh()
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)>


In [39]:
model.train()
# 优化器，反向传播后，还需要优化方法来更新网络的权重和参数
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # 可调参数 找到梯度下降的方向
criterion = nn.CrossEntropyLoss(weight=torch.Tensor([1,15]).to(device))

In [40]:
train_losses, train_accs = [], []
dev_losses, dev_accs = [], []

for epoch in range(50):
    train_loss, train_acc = 0, 0
    dev_loss, dev_acc = 0, 0
    
    # Training
    model.train()
    for trains, labels in train_iter:
        trains, labels = trains.cuda().float(), labels.long().cuda()
        outputs = model(trains)
        model.zero_grad() # 梯度清零
        loss = criterion(outputs, labels)
        loss.backward() # 反向传播
        optimizer.step() # 更新模型
        train_loss += loss.item()
        true = labels.data.cpu()
        predic = torch.max(outputs.data, 1)[1].cpu()
        train_acc += metrics.accuracy_score(true, predic)
    
    train_loss /= len(train_iter)
    train_acc /= len(train_iter)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Evaluation
    model.eval()
    with torch.no_grad():
        for dev_trains, dev_labels in test_iter:
            dev_trains, dev_labels = dev_trains.cuda().float(), dev_labels.long().cuda()
            dev_outputs = model(dev_trains)
            dev_loss += F.cross_entropy(dev_outputs, dev_labels).item()
            dev_true = dev_labels.data.cpu()
            dev_predic = torch.max(dev_outputs.data, 1)[1].cpu()
            dev_acc += metrics.f1_score(dev_true, dev_predic)
        
        dev_loss /= len(test_iter)
        dev_acc /= len(test_iter)
        dev_losses.append(dev_loss)
        dev_accs.append(dev_acc)
    
    # Print training and validation loss and accuracy
    print(f"Epoch [{epoch+1}/{50}]")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}",f"Dev Loss: {dev_loss:.4f}, Dev Acc: {dev_acc:.4f}")

Epoch [1/50]
Train Loss: 0.0992, Train Acc: 0.9677 Dev Loss: 0.8666, Dev Acc: 0.2166
Epoch [2/50]
Train Loss: 0.0803, Train Acc: 0.9623 Dev Loss: 0.7749, Dev Acc: 0.1879
Epoch [3/50]
Train Loss: 0.1094, Train Acc: 0.9636 Dev Loss: 0.7293, Dev Acc: 0.2480
Epoch [4/50]
Train Loss: 0.0819, Train Acc: 0.9545 Dev Loss: 0.6660, Dev Acc: 0.2220
Epoch [5/50]
Train Loss: 0.1044, Train Acc: 0.9605 Dev Loss: 0.6352, Dev Acc: 0.2087
Epoch [6/50]
Train Loss: 0.1386, Train Acc: 0.9418 Dev Loss: 0.5842, Dev Acc: 0.2448
Epoch [7/50]
Train Loss: 0.0884, Train Acc: 0.9518 Dev Loss: 0.5970, Dev Acc: 0.2736
Epoch [8/50]
Train Loss: 0.1260, Train Acc: 0.9482 Dev Loss: 0.5860, Dev Acc: 0.3442
Epoch [9/50]
Train Loss: 0.0807, Train Acc: 0.9491 Dev Loss: 0.6551, Dev Acc: 0.2461
Epoch [10/50]
Train Loss: 0.0674, Train Acc: 0.9641 Dev Loss: 0.7652, Dev Acc: 0.2896
Epoch [11/50]
Train Loss: 0.0653, Train Acc: 0.9645 Dev Loss: 0.7575, Dev Acc: 0.2312
Epoch [12/50]
Train Loss: 0.0717, Train Acc: 0.9627 Dev Loss: 0

In [47]:
# Assuming you have already created your DataLoader test_iter

all_true = []
all_predic = []

with torch.no_grad():
    for tests, labels in test_iter:
        tests, labels = tests.cuda().float(), labels.long().cuda()
        outputs = model(tests)  # Assuming model is your neural network
        true = labels.data.cpu()
        predic = torch.max(outputs.data, 1)[1].cpu()

        all_true.extend(true.data.numpy())
        all_predic.extend(predic.numpy())

# Calculate the classification report for the entire test dataset
classification_report = metrics.classification_report(all_true, all_predic, digits=4, output_dict=True)

# Convert the result to a DataFrame for better display
res = pd.DataFrame(classification_report).T
res


,precision,recall,f1-score,support
0,0.953086,0.910377,0.931242,848.000000
1,0.240000,0.387097,0.296296,62.000000
accuracy,0.874725,0.874725,0.874725,0.874725
macro avg,0.596543,0.648737,0.613769,910.000000
weighted avg,0.904503,0.874725,0.887982,910.000000


In [49]:
np.savez(f"../../output/baseline/ESM.npz", labels=test_labels, predictions=test_preds)